# Finding a Faster State-Checker for Tic-Tac-Toe

In [1]:
import numpy as np
from platform import python_version
python_version()

'3.7.1'

## 1. Test np.sum method

In [2]:
def check_game_state(state): 
    game_over, winner = False, None 
    for role in [1, 2]: 
        positions = (state == role) 
        if any(( 
                np.any(positions.sum(axis=0) == 3), 
                np.any(positions.sum(axis=1) == 3), 
                (np.diagonal(positions).sum() == 3), 
                (np.diagonal(np.fliplr(positions)).sum() == 3) 
        )): 
            game_over, winner = True, role 
            break
    if winner is None and np.all(state > 0): 
        game_over = True 
    return game_over, winner

In [3]:
state = np.random.randint(0, 3, size=9).reshape((3, 3))
state

array([[1, 0, 1],
       [2, 0, 2],
       [0, 0, 0]])

In [4]:
game_over, winner = check_game_state(state)
game_over, winner

(False, None)

## Generate some test results

In [5]:
game_states = []

print("Finding examples of game states...")
for case in [(False, None), # In progress
                          (True, 1),     # 1 won
                          (True, 2),     # 2 won
                          (True, None)   # Draw
                         ]:
    print(f" game_over, winner = {case}")
    for i in range(5):
        while check_game_state(state) != case: 
            state = np.random.randint(0, 3, size=9).reshape((3, 3))
        game_states.append((case, state))
len(game_states)

Finding examples of game states...
 game_over, winner = (False, None)
 game_over, winner = (True, 1)
 game_over, winner = (True, 2)
 game_over, winner = (True, None)


20

## 2. Test indexing method

This is based on this answer by Eric on stackoverflow.com:

https://stackoverflow.com/a/39185702/1609514

In [6]:
def product_slices(n): 
    for i in range(n): 
        yield ( 
            np.index_exp[np.newaxis] * i + 
            np.index_exp[:] + 
            np.index_exp[np.newaxis] * (n - i - 1) 
        )

def get_lines(n, k):
    """
    Returns:
        index (tuple):   an object suitable for advanced indexing to get all possible lines
        mask (ndarray):  a boolean mask to apply to the result of the above
    """
    fi = np.arange(k)
    bi = fi[::-1]
    ri = fi[:,None].repeat(k, axis=1)

    all_i = np.concatenate((fi[None], bi[None], ri), axis=0)

    # index which look up every possible line, some of which are not valid
    index = tuple(all_i[s] for s in product_slices(n))

    # We incrementally allow lines that start with some number of `ri`s, and an `fi`
    #  [0]  here means we chose fi for that index
    #  [2:] here means we chose an ri for that index
    mask = np.zeros((all_i.shape[0],)*n, dtype=np.bool)
    sl = np.index_exp[0]
    for i in range(n):
        mask[sl] = True
        sl = np.index_exp[2:] + sl

    return index, mask

In [7]:
n = 2  # Dimensions
k = 3  # size

# Prepare index and mask
index, mask = get_lines(n, k)

def check_game_state_indexing(state, n=n, k=k, 
                              index=index, mask=mask): 
    game_over, winner = False, None
    lines = state[index][mask]
    for role in [1, 2]: 
        if ((lines == role).sum(axis=1) == k).any():
            game_over, winner = True, role
            break
    if winner is None and np.all(state > 0): 
        game_over = True
    return game_over, winner

In [8]:
index

(array([[[0, 1, 2]],
 
        [[2, 1, 0]],
 
        [[0, 0, 0]],
 
        [[1, 1, 1]],
 
        [[2, 2, 2]]]), array([[[0, 1, 2],
         [2, 1, 0],
         [0, 0, 0],
         [1, 1, 1],
         [2, 2, 2]]]))

In [9]:
mask

array([[ True,  True,  True,  True,  True],
       [False, False, False, False, False],
       [ True, False, False, False, False],
       [ True, False, False, False, False],
       [ True, False, False, False, False]])

In [10]:
state

array([[2, 1, 1],
       [1, 2, 2],
       [2, 2, 1]])

In [11]:
state[index][mask]

array([[2, 2, 1],
       [1, 2, 2],
       [2, 1, 2],
       [1, 2, 2],
       [1, 2, 1],
       [2, 1, 1],
       [1, 2, 2],
       [2, 2, 1]])

In [12]:
# Test
check_game_state_indexing(state)

(True, None)

In [13]:
# Check results are identical
for case, state in game_states:
    assert check_game_state_indexing(state) == case
print("Testing successful!")

Testing successful!


## 3. Another 2 Variants

In [14]:
# Prepare sets of indices for each win line
xi = np.array([
    (0, 0, 0), 
    (1, 1, 1), 
    (2, 2, 2),
    (0, 1, 2), 
    (0, 1, 2),
    (0, 1, 2),
    (0, 1, 2),
    (0, 1, 2),
])

yi = np.array([
    (0, 1, 2), 
    (0, 1, 2), 
    (0, 1, 2),
    (0, 0, 0), 
    (1, 1, 1), 
    (2, 2, 2),
    (0, 1, 2),
    (2, 1, 0)
])

def check_game_state_tuple_indexing(state, xi=xi, yi=yi): 
    game_over, winner = False, None
    lines = state[xi, yi]
    for role in [1, 2]: 
        if ((lines == role).sum(axis=1) == 3).any():
            game_over, winner = True, role
            break
    if winner is None and np.all(state > 0): 
        game_over = True
    return game_over, winner

In [15]:
# Prepare 3D masks of all possible win lines for each player
mask_arrays = {}
for role in [1, 2]:
    mask_array = -np.ones((8, 3, 3), dtype=int)  # Start with -1 values
    for i, (x, y) in enumerate(zip(xi, yi)):
        mask_array[i, x, y] = role
    mask_arrays[role] = mask_array

def check_game_state_array_mask(state, mask_arrays=mask_arrays): 
    game_over, winner = False, None
    for role in [1, 2]:
        if ((state == mask_arrays[role]).sum(axis=(1, 2)) == 3).any():
            game_over, winner = True, role
            break
    if winner is None and np.all(state > 0): 
        game_over = True
    return game_over, winner

In [16]:
check_game_state_tuple_indexing(state)

(True, None)

In [17]:
# Test
check_game_state_array_mask(state)

(True, None)

In [18]:
# Check results are identical
for case, state in game_states:
    assert check_game_state_tuple_indexing(state) == case
    assert check_game_state_array_mask(state) == case
print("Testing successful!")

Testing successful!


## 3. Speed testing

In [19]:
%timeit [check_game_state(state) for case, state in game_states]

1e+03 µs ± 40.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%timeit [check_game_state_indexing(state) for case, state in game_states]

403 µs ± 4.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
%timeit [check_game_state_tuple_indexing(state) for case, state in game_states]

332 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%timeit [check_game_state_array_mask(state) for case, state in game_states]

330 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
